# Représentation Bag-of-Words et Régression Logistique

In [ ]:
# On importe les librairies nécessaires pour ce TD,
# dont le second bloc est tiers, i.e. pas installé
# avec Python par défaut.
# Localement, on aura donc recours à pip (dans la console
# système et non la console python) pour les installer au
# besoin (instructions en commentaire ci-dessous).

import re  # librairie standard (i.e. livrée avec Python)

import matplotlib.pyplot as plt  # pip install matplotlib>=3.2
import numpy as np   # pip install numpy>=1.15
import pandas as pd  # pip install pandas>=1.0
import scipy         # pip install scipy>=1.4
import sklearn       # pip install scikit-learn>=0.23

#### Présentation des données

On va utiliser un jeu de données open-source qui n'a rien à voir avec la santé : il s'agit de critiques de films sur IMDB, en anglais, accompagnées d'une note. Ces dernières vont normalement de 1 à 10, mais les notes égales à 5 et à 6 ont été exclues, afin de définir une variable de polarité : avis positif (>= 7) ou négatif (<= 4).

L'objet va donc être de manipuler un peu ces données pour en construire une représentation numérique adaptée à la mise en oeuvre de modèles d'apprentissage supervisé pour une tâche de classificaiton binaire : la prédiction de la polarité de la critique, une tâche classique d'analyse de sentiment.

Nous disposons d'un total de 50 000 échantillons, que j'ai répartis en trois jeux :
* train : 35 000 (70%) échantillons, utilisés pour entraîner des modèles de classification binaire
* valid : 12 500 (15%) échantillons, utilisés pour évaluer les modèles et en chercher les meilleurs hyper-paramètres
* test  : 12 500 (15%) échantillons, utilisés pour l'évaluation finale du ou des modèles retenus

source des données : [Large Movie Review Dataset v1.0](http://ai.stanford.edu/%7Eamaas/data/sentiment/)<br/>
le notebook de préparation des données est fourni pour votre curiosité

#### Chargement des données

Les données sont stockées au format tsv (_tab-separated values_) ; on va utiliser [Pandas](https://pandas.pydata.org/) pour les lire et les manipuler.

In [ ]:
train = pd.read_csv('data/train.tsv', sep='\t')

In [ ]:
# Affiche les premières lignes (les textes seront tronqués).
train.head()

In [ ]:
# Visualisation de la distribution des notes.
%matplotlib inline  # spécifique à Jupyter Notebook

train['rate'].value_counts().sort_index().plot.bar(
    title='Effectifs des échantillons par note.'
)

In [ ]:
# Part des échantillons d'entraînement positifs.
print(train['polarity'].mean())

#### Exemples de manipulation de string

Prenons un unique texte et manipulons-le un peu.

In [ ]:
text = train.loc[2, 'text']
print(text)

In [ ]:
# Passage en minuscule.
print(text.lower())

In [ ]:
# Retrait de la ponctuation (sauf tirets, apostrophes et barres obliques).
print(re.sub(r"[^\w\s\-'/]", "", text))

In [ ]:
# Retrait de certains mots peu informatifs (stopwords).
# On voit qu'établir une liste est vite pénible !
print(' '.join(
    word for word in text.lower().split(' ')
    if word not in ('i', 'in', 'it', 'at', 'but', 'and', 'for', 'or', 'the', 'a', 'is', 'are', 'were')
))

In [ ]:
# Extraction de tous les groupes de trois mots (3-grams) non séparés par de la ponctuation.

# (a) on crée une copie normalisée du texte
tbis = re.sub(r"[^\w\s\.\-'/]", "", text.lower())  # on retire la ponctuation indésirable.
tbis = tbis.replace('?', '.').replace('!', '.')  # on remplace certains signes
tbis = re.sub('  +', ' ', tbis)  # on retire les espaces redondants
tbis = re.sub(r'\.\.+', '\.', tbis)  # on retire les points redondants
tbis = tbis.replace(' .', '.')  # on retire les espaces avant un point
print(tbis)

# (b) on découpe en phrases ; pour chaque phrase on produit les trigrammes.
trigrams = []
for sentence in tbis.split('.'):
    tokens = sentence.strip(' ').split(' ')
    for i in range(len(tokens) - 2):
        trigrams.append(
            (tokens[i], tokens[i + 1], tokens[i + 2])
        )
print(trigrams)

#### Systématisons un peu

Ici, on va :
* définir et appliquer des premières règles de nettoyage pour tous les textes
* explorer le vocabulaire des textes nettoyés et la fréquence des termes qui le composent
* choisir un vocabulaire restreint à partir de critères de fréquence

In [ ]:
# Normalisons un peu nos textes.

def normalize_text(text):
    """Apply basic normalization to a given text."""
    tbis = re.sub(r"[^\w\s\.\-'/]", "", text.lower())  # on retire la ponctuation indésirable.
    tbis = tbis.replace(' - ', ' ')  # on ne garde que les tirets intra-mot
    tbis = tbis.replace('?', '.').replace('!', '.')  # on remplace certains signes
    tbis = re.sub('  +', ' ', tbis)  # on retire les espaces redondants
    tbis = re.sub(r'\.\.+', '\.', tbis)  # on retire les points redondants
    tbis = tbis.replace(' .', '.')  # on retire les espaces avant un point
    return tbis.strip('. ') + '.'   # on oblige à commencer par une lettre et finir par un point


train['clean'] = train['text'].apply(normalize_text)

In [ ]:
# Explorons notre vocabulaire.
# On va répertorier tous les mots contenus dans les textes, ainsi que leur nombre d'occurence.

# Définissons une fonction pour ce faire.
def get_vocab_count(series):
    """List terms and their number of occurences for a pandas.Series of texts."""
    # On définit un dictionaire (vide) pour contenir les couples ('mot': compte)
    vocab = {}
    # On définit une fonction qui met à jour le vocabulaire pour un texte donné.
    # (jargon technique: cette fonction est une closure, qui a un effet de bord)
    def count_vocab(text):
        nonlocal vocab
        for token in text.replace('.', ' ').split(' '):
            token = token.strip(' ')
            if token:  # on ne prend pas les chaînes vides
                count = vocab.get(token, 0)  # compte actuel, ou 0 si nouveau terme
                vocab[token] = count + 1     # mise à jour du compte (ou création)
    # On applique la fonction à notre pandas.Series.
    # Note: c'est plus rapide que de faire une boucle sur les textes.
    series.apply(count_vocab)  # l'effet de bord met à jour 'vocab'
    # On retourne le vocabulaire qu'on a construit, comme une pandas.Series.
    return pd.Series(vocab)


# Appliquons à nos textes (nettoyés).
vocab = get_vocab_count(train['clean'])

In [ ]:
print("Nombre de termes distincts : %s" % len(vocab))
print("Nombre de termes apparaissant plus de 1 fois : %s" % (vocab > 1).sum())
print("Nombre de termes apparaissant plus de 10 fois : %s" % (vocab > 10).sum())
print("Nombre de termes apparaissant plus de 100 fois : %s" % (vocab > 100).sum())

Visualisons l'histogramme de la (log-)fréquence des mots :

In [ ]:
_, axes = plt.subplots(1, 2, figsize=(15, 5))
np.log(vocab).plot.hist(ax=axes[0], title='all words')
np.log(vocab[vocab > 1]).plot.hist(ax=axes[1], title='words appearing 2+ times')

Vous connaissez la [loi de Zipf](https://fr.wikipedia.org/wiki/Loi_de_Zipf) ?
Ici, c'est pire !

In [ ]:
plt.plot(vocab.sort_values(ascending=False).values[:1000])
plt.title("Nombre d'occurrence des 1000 tokens les plus fréquents selon leur rang.")

In [ ]:
# Regardons les vingt mots les plus fréquents. C'est peu édifiant.
vocab.sort_values(ascending=False)[:20]

### Représentation bag-of-words

On s'est échauffés en faisant des choses manuellement, mais utilisons plutôt un outil clefs-en-main, qui sera plus performant, pour aller plus loin.

On va utiliser la librairie [Scikit-Learn](https://scikit-learn.org/), qui nous fournira également par la suite des modèles d'apprentissage supervisé (hors réseaux de neurones). C'est une librairie très complète, mais surtout très bien documentée, et il y a beaucoup à apprendre sur l'analyse de données et le _machine learning_ sur leur site, qui se veut à la fois précis et pédagogue.

On aurait presque pu remplacer ce tutoriel par [le leur](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html) ; d'ailleurs notre sujet est suggéré en exercice...

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

Qu'est-ce qu'on fait quand on a un bel outil dont on veut se servir mais qu'on ne connaît pas ? On lit [la documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) !

In [ ]:
# dans la console standard, on ferait help(CountVectorizer)
?CountVectorizer

En résumé, on peut :
* retirer les acccents, passer en lowercase (True par défaut), utiliser des stopwords...
* limiter la taille du vocabulaire, ou forcer l'usage d'un vocabulaire pré-défini
* utiliser des n-grams pour n dans un intervalle donné (par défaut, unigrammes seulement)
* changer la façon dont les tokens sont découpés (par défaut, deux lettres ou plus séparées par tout autre symbole ou espace)
* générer des sacs de mots binaires (par défaut, on compte les occurences)
* fixer des limites à la part de documents du corpus dans laquelle un token apparaît pour ne pas être écarté

In [ ]:
# On instancie un object 'CountVectorizer', paramétré comme il nous sied.
count_vect = CountVectorizer(
    lowercase=True,           # on passe le texte en minuscule
    stop_words='english',     # liste existante de stopwords anglais à filtrer
    token_pattern=r"\b[\w\-']+\b",  # on ne coupe pas les unigrammes autour des tirets et apostrophes
    ngram_range=(1, 2),       # on considère les unigrammes et les bigrammes
    min_df=(100 / len(train)) # on ne conserve que les tokens apparaissant dans au moins cent documents
)

In [ ]:
# On établit le vocabulaire, puis on produite la matrice documents-termes.
# Ceci peut prendre un peu de temps, malgré le beau travail d'optimisation derrière.
dtm = count_vect.fit_transform(train['clean'])

La DTM est sparse ; on peut néanmoins accéder à ses contenus.<br/>
On va s'intéresser au vocabulaire établi, au nombre total d'occurences des tokens, et au nombre de documents distincts dans lesquels ils figurent.

A noter : par convention, pour tous les modèles implémentés dans Scikit-Learn, les attributs dont le nom finit par '\_' sont ceux créés pendant l'appel à la méthode `fit` de l'objet.

In [ ]:
# On récupère le dictionnaire {terme: indice dans le vocabulaire} et on le renverse.
vocab = {
    idx: token for token, idx in count_vect.vocabulary_.items()
}

print("Number of tokens in the vocabulary: %s" % len(vocab))
print("Out of which %s are bigrams." % sum(' ' in token for token in vocab.values()))

In [ ]:
# Construisons un DataFrame avec des informations sur le vocabulaire.
voc_df = pd.DataFrame({'token': vocab})
voc_df['bigram'] = voc_df['token'].str.contains(' ')
voc_df['counts'] = np.array(dtm.sum(axis=0))[0]
voc_df['n_docs'] = np.array((dtm > 0).sum(axis=0))[0]

voc_df.tail()

In [ ]:
# Au passage, la DTM est-elle vraiment sparse ?
# On quantifie la part de valeurs non-nulles:
(dtm > 0).sum() / (dtm.shape[0] ** 2)

In [ ]:
# Quels sont les vingts tokens les plus fréquents ?
voc_df.sort_values('counts', ascending=False).iloc[:20]

In [ ]:
# Quels sont les vingts tokens apparaissant dans le plus de documents ?
voc_df.sort_values('n_docs', ascending=False).iloc[:20]

In [ ]:
# Quels sont les bigrammes les plus fréquents ?
voc_df[voc_df['bigram']].sort_values('counts', ascending=False).iloc[:20]

### Analyse de sentiment

#### Tests du chi-deux

On va commencer par un test statistique simple, pour déterminer pour chacun des tokens retenus s'il y a une différence significative de sa fréquence selon que la critique est positive ou non.

In [ ]:
# On extrait les indices des échantillons positifs et négatifs.
pos = train.loc[train['polarity'] == 1].index
neg = train.loc[train['polarity'] == 0].index
# Pour chaque groupe, pour chaque token, on collecte
# le nombre de documents dans lesquels il apparaît.
voc_df['pos_docs'] = np.array((dtm[pos] > 0).sum(axis=0))[0]
voc_df['neg_docs'] = np.array((dtm[neg] > 0).sum(axis=0))[0]

voc_df.tail()

In [ ]:
# On effectue, pour chaque mot, le test du chi-deux.
from scipy.stats import chisquare  # third-party package (`pip install scipy`)

voc_df['pvalue'] = chisquare(voc_df[['pos_docs', 'neg_docs']].values, axis=1).pvalue.round(5)

voc_df.tail()

print("Share of p-values below 0.05: %s" % (voc_df['pvalue'] < 0.05).mean().round(4))
print("Share of p-values below 0.01: %s" % (voc_df['pvalue'] < 0.01).mean().round(4))
print("Share of p-values below 0.001: %s" % (voc_df['pvalue'] < 0.001).mean().round(4))

In [ ]:
voc_df[voc_df['pvalue'] > 0.05]

In [ ]:
voc_df[(voc_df['pvalue'] < 0.001) & (voc_df['pos_docs'] > voc_df['neg_docs'])].sort_values('pos_docs')

In [ ]:
voc_df[(voc_df['pvalue'] < 0.001) & (voc_df['pos_docs'] < voc_df['neg_docs'])].sort_values('neg_docs')

#### Régression logistique

On entraîne une régression logistique pénalisée par LASSO sur nos données.

In [ ]:
from sklearn.linear_model import LogisticRegression
# On instancie le modèle; les options servent à utiliser la pénalisation LASSO.
lreg = LogisticRegression(penalty='l1', solver='liblinear')
# On entraîne le modèle sur nos données.
lreg.fit(dtm, train['polarity'])

On évalue le modèle sur le données d'entraînement.

In [ ]:
# Raccourci pour l'accuracy du modèle.
lreg.score(dtm, train['polarity'])

In [ ]:
# Si on veut faire les choses manuellement :
y_true = train['polarity']
y_pred = lreg.predict(dtm)

print("True Positives : %s" % y_pred[y_true == 1].sum())
print("True Negatives : %s" % (1 - y_pred[y_true == 0]).sum())
print("False Positives : %s" % y_pred[y_true == 0].sum())
print("False Negatives : %s" % (1 - y_pred[y_true == 1]).sum())

In [ ]:
# Mais scikit-learn fournit déjà les outils.

print(sklearn.metrics.classification_report(y_true, y_pred))

Mais évaluons maintenant sur le jeu de validation :

In [ ]:
# On charge les données de validation et on en produit la DTM
# (selon le même vocabulaire de tokens).
valid = pd.read_csv('data/valid.tsv', sep='\t')
valid['clean'] = valid['text'].apply(normalize_text)
valid_dtm = count_vect.transform(valid['clean'])

In [ ]:
# On évalue. C'est correct, mais moins bien!
y_true = valid['polarity']
y_pred = lreg.predict(valid_dtm)

print(sklearn.metrics.classification_report(y_true, y_pred))

Nous avons un premier modèle, imparfait, qui peut nous servir de _baseline_ : nos efforts par la suite porteront sur le fait d'aller au-delà de ces performances, qui servent de référence de départ.

## Sujet ouvert : comment faire mieux ?

#### Premier exemple : sous-séléction informée des variables

Une idée parmi d'autres : se restreindre aux termes qui semblent associés à l'output au sens du test d'indépendance du chi-deux.

In [ ]:
# On entraîne une nouvelle régression, en réduisant le vocabulaire
# sur la base de nos tests du chi-deux.
lreg = LogisticRegression(penalty='l1', solver='liblinear')

idx = voc_df[voc_df['pvalue'] < 0.001].index.values
lreg.fit(dtm[:, idx], train['polarity'])

In [ ]:
print("Evaluation sur les données d'entraînement :")
print(sklearn.metrics.classification_report(
    y_true=train['polarity'],
    y_pred=lreg.predict(dtm[:, idx])
))

In [ ]:
print("Evaluation sur les données de validation :")
print(sklearn.metrics.classification_report(
    y_true=valid['polarity'],
    y_pred=lreg.predict(valid_dtm[:, idx])
))

Question : Comment interpréter ce petit changement dans les résultats ?

In [ ]:
?LogisticRegression

#### Des idées en pagaille :

Modifier les features :
* étendre le vocabulaire à des n-grams plus longs
* modifier les coefficients de la DTM : valeurs binaires ou valeurs TF-IDF (avec `sklearn.feature_extraction.text.TfidfTransformer`)
* modifier les critères de sélection parmi le vocabulaire complet

Modifier les hyper-paramètres du modèle :
* essayer une autre pénalité que LASSO (l1)
* modifier le coefficient de régularization (paramètre `C`)

Modifier la classe du modèle :
* arbre de décision (`sklearn.tree.DecisionTreeClassifier`)
* RandomForest (`sklearn.ensemble.RandomForestClassifier`)
* SVM (`sklearn.svm.SVC` ou `sklearn.svm.LinearSVC`)

On peut évidemment combiner ces différents points (jouer sur les features _et_ la classe du modèle _et_ ses hyper-paramètres). Comment faire cela de manière un tant soit peu ordonnée ?

#### Un peu de méthodologie :

Un peu de vocabulaire essentiel en _Machine Learning_, mais un peu au-delà de ce TP.<br/>
Aujourd'hui on pourra s'autoriser à opérer de manière moins structurée, mais ce sont de bonnes ressources en pratique.<br/>
Un exemple d'utilisation est proposé dans le notebook "2_sandbox_template", que je vous invite à dupliquer et modifier de fond en comble pour mettre en place vos propres chaînes de pré-traitement et d'entraînement.

* Validation croisée ([tutoriel scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html))
    * intuition : on entraîne plusieurs fois un même modèle sur des sous-échantillons aléatoires, pour avoir une meilleure estimation de sa capacité à généraliser sur de nouvelles données
* Grid Search ([tutoriel scikit-learn](https://scikit-learn.org/stable/modules/grid_search.html))
    * intuition : on répète une même procédure d'entraînement, avec des hyper-paramètres distincts, pour trouver la meilleure combinaison de ces paramètres pour une classe de modèle et un jeu de données fixés

#### Et le word embedding?

On y vient, dans un notebook distinct!